<a href="https://colab.research.google.com/github/tiagoenriquez/conversa-com-ia/blob/main/Conversa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Audio, display, Javascript

language = "pt"

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=10):
  print("Ouvindo...")
  display(Javascript(RECORD))
  result = output.eval_js('record(%d)' % (sec*1000))
  audio = b64decode(result.split(',')[1])
  nome_do_arquivo = "request_audio.wav"
  with open(nome_do_arquivo, "wb") as arquivo:
    arquivo.write(audio)
  return f"/content/{nome_do_arquivo}"

record_file = record()
display(Audio(record_file, autoplay=False))

Ouvindo...


<IPython.core.display.Javascript object>

In [3]:
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 8.0 MB/s eta 0:00:00


In [4]:
import whisper

model = whisper.load_model("medium")
result = model.transcribe(record_file, fp16=False, language=language)
pergunta = result["text"]
print(pergunta)

100%|██████████████████████████████████████| 1.42G/1.42G [00:06<00:00, 250MiB/s]


 Como ser aprovado no Enem?


In [5]:
pip install gpt4all

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.1 MB/s eta 0:00:00


In [6]:
from gpt4all import GPT4All

model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf")
resposta = ""
with model.chat_session():
  resposta = model.generate(pergunta, max_tokens=1024)
  print(resposta)

Downloading: 100%|██████████| 4.66G/4.66G [01:14<00:00, 62.5MiB/s]
Verifying: 100%|██████████| 4.66G/4.66G [00:27<00:00, 171MiB/s]


O ENEM (Exame Nacional do Ensino Médio) é um exame importante para os estudantes brasileiros que concluem o ensino médio. Para ser aprovado, você precisa atingir uma nota mínima de 400 pontos em cada disciplina e obter ao menos 550 pontos no total. Aqui estão algumas dicas para ajudá-lo a se preparar e aumentar suas chances de aprovação:

1. **Entenda o formato do exame**: O ENEM é composto por quatro provas: Língua Portuguesa, Matemática, Ciências Naturais e Redação.
2. **Aprenda os conteúdos**: Revise os tópicos estudados durante o ensino médio em cada disciplina. Isso ajudará a você entender melhor as questões do exame.
3. **Pratique com materiais de estudo**: Utilize livros, apontamentos e recursos online para se preparar. Faça exercícios e testes para avaliar seu conhecimento.
4. **Desenvolva habilidades específicas**: Em Língua Portuguesa, pratique a compreensão de textos, análise literária e redação; em Matemática, desenvolva suas habilidades com equações, funções e estatística.

In [7]:
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1


In [8]:
from gtts import  gTTS

gtts_object = gTTS(text=resposta, lang=language, slow=False)
response_audio = "/content/response_audio.wav"
gtts_object.save(response_audio)
display(Audio(response_audio, autoplay=True))